# Pet-project Oyster

## Описание проекта

TODO

https://habr.com/ru/sandbox/197522/
https://proverkacheka.com/cabinet/checks

## Описание данных

TODO  
qrraws.csv  
Bills_(year).csv  
Goods_(year).csv  

## Импорт библиотек

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

import requests
import json
import uuid
import datetime
import os


## Константы

In [2]:
URL = "https://proverkacheka.com/api/v1/check/get"
TOKEN = "25602.ZzjFe81k1NGymEnRU"
QRRAWS_PATH = "./sources/qrraws.csv"
SOURCES_DIR = "./sources"
# BILLS_SOURCE_NAME = "Bill"
# GOODS_SOURCE_NAME = ""

## Вспомогательные методы

### Для подгрузки данных

In [3]:
# ОТЛАЖЕНО
def load_qrraws_data():
    """
    Метод, который выгружает qrraws_data и возвращает df 
    и выводит справочнуюинформацию о нём
    """
    
    qrraws_data = pd.read_csv(QRRAWS_PATH, sep=';')
    qrraws_data = qrraws_data.set_index('id')
    
    # Вывод справочной информации
    print(qrraws_data.head(3))
    qrraws_data.info()
    
    return qrraws_data

In [4]:
# ОТЛАЖЕНО
def get_bill_data(qrraw, state):
    """
    Метод отправляет обращения на сервис проверки чеков, если по этому чеку ещё нет записи в Bills_year.csv (state != 1),
    после этого, если всё хорошо (код ответа == 1), обращается к методу записи результата в Bills_year.csv. 
    При любом исходе метод возвращает новый state для текущего qrraw.
    
    Пример qrraw:
    "t=20200924T1837&s=349.93&fn=9282440300682838&i=46534&fp=1273019065&n=1"
    """
    
    if state == 1:
        return state
    
    url = URL
    data = {
        "token": TOKEN,
        "qrraw": qrraw,
    }
    r = requests.post(url, data=data)
    result = json.loads(r.text)
    
    if result['code'] != 1:
        print(f"ERR {result['code']}, {result['data']} qrraw = {qrraw}")
    else:
        write_bill_data(result, r.text)
    return result['code']

# TODO проработать расшифровку кодов ответов. Обрабатываем по-новой те записи, где код не 1
# ERR 3, Превышено количество обращений по чеку.
# ERR 3, Превышено количество обращений для учетной записи пользователя.
# ERR 4, Не вышло время ожидания перед повторным запросом.
# ERR 5, Нет информации по чеку (прочее).

In [5]:
# ОТЛАЖЕНО
def write_bill_data(result, text_version):
    """
    Метод добавляет запись в таблицу Bills_year.csv. Происходит автоматическое создание файла по году, если такого ещё нет.
    При первичном создании в bill_data.to_csv header=True, при последующих False, чтобы не дублировались заголовки.
    Метод возвращает 200, если в процессе работы не произошло ошибок.
    """
    
    records_year = pd.to_datetime(result['data']['json']['dateTime'], format='%Y-%m-%dT%H:%M:%S').year
    record_uuid = uuid.uuid4()
    qrraw = result['request']['qrraw']
    scan_date = pd.to_datetime(datetime.date.today().isoformat(), format='%Y-%m-%d')
    day_of_purchase = pd.to_datetime(result['data']['json']['dateTime'], format='%Y-%m-%dT%H:%M:%S')
    
    bill_data = pd.DataFrame(
        [[record_uuid, qrraw, text_version, scan_date, day_of_purchase, 0]], 
        columns=['id', 'qrraw', 'full_data', 'scan_date', 'day_of_purchase', 'parsed']
    )
    file_path = f'./sources/Bills_{records_year}.csv'
    header_flag = check_file(file_path) == False
    
    bill_data.to_csv(file_path, mode='a', header=header_flag, index=False)
    return 200

In [6]:
# ОТЛАЖЕНО
def check_file(file_path):
    """
    Метод проверяет, существует ли файл по указанному пути. Возвращает, соответственно True или False.
    """
    
    if os.path.exists(file_path):
        return True
    else:
        return False

In [7]:
def load_goods_data(bill_id, full_data, parsed):
    """
    TODO
    """
    if parsed == 1:
        return parsed
    
    full_data = json.loads(full_data)
    records_year = pd.to_datetime(full_data['data']['json']['dateTime'], format='%Y-%m-%dT%H:%M:%S').year
    shop_name = full_data['data']['json']['retailPlace']
    shop_address = full_data['data']['json']['metadata']['address']
    day_of_purchase = pd.to_datetime(full_data['data']['json']['dateTime'], format='%Y-%m-%dT%H:%M:%S')
    load_date = pd.to_datetime(datetime.date.today().isoformat(), format='%Y-%m-%d')
    
    for item in full_data['data']['json']['items']:
        name = item['name']
        price = item['price']
        quantity = item['quantity']
        full_cost = item['sum']
        category_id = detect_category(name) # TODO в разработке отдельный метод-классификатор, пока возвращает 0 всегда
        
        good_data = pd.DataFrame(
            [[bill_id, name, price, quantity, full_cost, shop_name, shop_address, day_of_purchase, load_date, category_id]], 
            columns=['bill_id', 'name', 'price', 'quantity', 'full_cost', 'shop_name', 'shop_address', 'day_of_purchase', 'load_date', 'category_id']
        )
        file_path = f'./sources/Goods_{records_year}.csv'
        header_flag = check_file(file_path) == False
        
        good_data.to_csv(file_path, mode='a', header=header_flag, index=False)
    return 1

In [8]:
def start_goods_loader():
    """
    TODO
    """
    # Получаем список файлов
    files = os.listdir(SOURCES_DIR)
    found = [file for file in files if 'Bills' in file]
    years_list = list(map(lambda x: x.split('.')[0].split('_')[1], found))
    
    for year in years_list:
        bills_data = pd.read_csv(f'./sources/Bills_{year}.csv')
        display(f'Обработка данных из Bills_{year}.csv')
        display(bills_data.head(3))
    
        # Запуск загрузки в Goods_year.csv
        bills_data['parsed'] = bills_data.apply(lambda x: load_goods_data(x.id, x.full_data, x.parsed), axis=1)
        
        # Перезапись данных в Bills_year.csv, туда отправляются обновлённые коды стейта загрузки в Goods_year.csv
        bills_data.to_csv(f'./sources/Bills_{year}.csv', header=True, index=False)

In [9]:
def detect_category(good_name):
    """
    TODO
    """
    #TODO
    return 0

### Для аналитики

## Раздел для ручного добавления в qrraws.csv 

Пока не реализовано ничего для автоматизации пополнения qrraws, добавляем записи в полуручном режиме:  
- вручную или через сканер получить qrraw-ы с чеков ([Для сканирования qr с компьютера](#https://code-qr.ru/online))
- записать эти qrraw-ы в массив new_qrraws ниже, как строки

In [10]:
# Тут формируем массив с новыми qrraw. Если внести qrraw, который уже есть в таблице, дубля не будет, т.к. метод его пропустит
new_qrraws = [
't=20240118T1612&s=345.30&fn=7282440500188699&i=68595&fp=3601964778&n=1',
't=20240117T2120&s=120.62&fn=7282440500188723&i=58167&fp=3457433757&n=1',
]

In [11]:
# Вытаскиваем уже имеющиеся в источнике данные
qrraws_data = load_qrraws_data()

# Формирование валидного массива словарей из new_qrraws 
arr_of_additions = []
for new_qrraw in new_qrraws:
    if qrraws_data.query("qrraws == @new_qrraw")['qrraws'].count() > 0:
        print(f'Уже есть в qrraws: {new_qrraw}')
    else:
        arr_of_additions.append({'qrraws':new_qrraw, 'state':0})

# Преобразование массива словарей new_qrraws в df и присоединение его к основному qrraws_data
addition = pd.DataFrame(arr_of_additions)
qrraws_data=qrraws_data.append(addition,ignore_index=True)

# Профилактическая очистка от дубликатов qrraw в qrraws_data
qrraws_data = qrraws_data.drop_duplicates(['qrraws'])
# Запись новых данных в источник qrraws
qrraws_data.to_csv(QRRAWS_PATH, sep=';', index=True, index_label='id')

# Проверка, остались ли дубликаты
qrraws_data[qrraws_data.duplicated(['qrraws'])]

                                               qrraws  state
id                                                          
0   t=20230809T2200&s=328.67&fn=9960440503006980&i...      1
1   t=20230809T2318&s=227.82&fn=9960440302365866&i...      1
2   t=20230805T2105&s=1992.90&fn=9960440302365866&...      1
<class 'pandas.core.frame.DataFrame'>
Int64Index: 222 entries, 0 to 221
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   qrraws  222 non-null    object
 1   state   222 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 5.2+ KB
Уже есть в qrraws: t=20240118T1612&s=345.30&fn=7282440500188699&i=68595&fp=3601964778&n=1
Уже есть в qrraws: t=20240117T2120&s=120.62&fn=7282440500188723&i=58167&fp=3457433757&n=1


,qrraws,state


## Раздел подгрузки данных

**Запуск выгрузки данных о чеках в Bills_year.csv по qrraws**. Запросы идут только на записи, где state !=1, иначе считаем, что по этому чеку уже есть данные в Bills_year.csv. Тут же будет вывод о qrraws, для которых не получилось добавить данные по чекам из-за ошибок:

In [12]:
# Считываем данные из хранилища qrraws
qrraws_data = load_qrraws_data()

# ****************************************************************************************
# На случай, если нужно полностью восстановить чеки из qrraws, сбрасываем стейты
# qrraws_data['state'] = 0
# qrraws_data.to_csv(QRRAWS_PATH, sep=';', index=True, index_label='id')
# ****************************************************************************************

# Запуск загрузки в Bills_year.csv
qrraws_data['state'] = qrraws_data.apply(lambda x: get_bill_data(x.qrraws, x.state), axis=1)
# Перезапись данных в qrraws.csv, туда отправляются обновлённые коды стейта загрузки в Bills_year.csv
qrraws_data.to_csv(QRRAWS_PATH, sep=';', index=True, index_label='id')

                                               qrraws  state
id                                                          
0   t=20230809T2200&s=328.67&fn=9960440503006980&i...      1
1   t=20230809T2318&s=227.82&fn=9960440302365866&i...      1
2   t=20230805T2105&s=1992.90&fn=9960440302365866&...      1
<class 'pandas.core.frame.DataFrame'>
Int64Index: 222 entries, 0 to 221
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   qrraws  222 non-null    object
 1   state   222 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 5.2+ KB
ERR 3, Превышено количество обращений по чеку. qrraw = t=20231029T1838&s=909.09&fn=7281440501305730&i=3739&fp=2449765930&n=1


In [13]:
# ****************************************************************************************
# На случай, если нужно убрать записи, где код не 0 или не 1
# qrraws_data = qrraws_data.query("state == 0 or state == 1")
# qrraws_data = qrraws_data.reset_index(drop= True)
# qrraws_data
# qrraws_data.to_csv(QRRAWS_PATH, sep=';', index=True, index_label='id')
# ****************************************************************************************

*При повторных запусках кода выше не будет никаких лишних дублей*

**Запуск выгрузки данных о чеках в Goods_year.csv по Bills_year.csv**. Запросы идут только на записи, где state !=1, иначе считаем, что по этому чеку уже есть данные в Goods_year.csv.

In [14]:
start_goods_loader()

'Обработка данных из Bills_2024.csv'

,id,qrraw,full_data,scan_date,day_of_purchase,parsed
0,896b7f48-a3a9-45d6-8723-8090c60a29b5,t=20240115t1114&s=108.28&fn=7282440500188699&i...,"{""code"":1,""first"":0,""data"":{""json"":{""code"":3,""...",2024-01-19 00:00:00,2024-01-15 11:14:00,1
1,7f4c0341-db6e-4d94-bd57-1031e4dc6cc5,t=20240114t0108&s=339.11&fn=7282440500188527&i...,"{""code"":1,""first"":0,""data"":{""json"":{""code"":3,""...",2024-01-19 00:00:00,2024-01-14 01:08:00,1
2,ceb15763-b8ca-4053-9951-6a80c05a1715,t=20240110t0245&s=256.78&fn=7282440500188699&i...,"{""code"":1,""first"":0,""data"":{""json"":{""code"":3,""...",2024-01-19 00:00:00,2024-01-10 02:45:00,1


'Обработка данных из Bills_2023.csv'

,id,qrraw,full_data,scan_date,day_of_purchase,parsed
0,be4662ee-7b31-4d77-aca9-5c102a008029,t=20230809t2200&s=328.67&fn=9960440503006980&i...,"{""code"":1,""first"":0,""data"":{""json"":{""code"":3,""...",2024-01-19 00:00:00,2023-08-09 22:00:00,1
1,ca0da635-c7ff-4eda-a6bd-683745e5232e,t=20230809t2318&s=227.82&fn=9960440302365866&i...,"{""code"":1,""first"":0,""data"":{""json"":{""code"":3,""...",2024-01-19 00:00:00,2023-08-09 23:18:00,1
2,3b0f9f35-f908-4849-adfd-66604e7af629,t=20230805t2105&s=1992.90&fn=9960440302365866&...,"{""code"":1,""first"":0,""data"":{""json"":{""code"":3,""...",2024-01-19 00:00:00,2023-08-05 21:05:00,1


In [15]:
# Получаем список файлов
files = os.listdir(SOURCES_DIR)
found = [file for file in files if 'Goods' in file]
years_list = list(map(lambda x: x.split('.')[0].split('_')[1], found))

full_df = pd.DataFrame()

for year in years_list:
    goods_data = pd.read_csv(f'./sources/Goods_{year}.csv')
    display(f'Обработка данных из Goods_{year}.csv')
    display(goods_data.head(1))
    display(goods_data.info())
    full_df = full_df.append(goods_data,ignore_index=True)

'Обработка данных из Goods_2023.csv'

,bill_id,name,price,quantity,full_cost,shop_name,shop_address,day_of_purchase,load_date,category_id
0,be4662ee-7b31-4d77-aca9-5c102a008029,MAXIDUO Страчателла,5889,1.0,5889,"магазин ""Красное & Белое""","117041,Россия,город федерального значения Моск...",2023-08-09 22:00:00,2024-01-19 00:00:00,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 935 entries, 0 to 934
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   bill_id          935 non-null    object 
 1   name             935 non-null    object 
 2   price            935 non-null    int64  
 3   quantity         935 non-null    float64
 4   full_cost        935 non-null    int64  
 5   shop_name        935 non-null    object 
 6   shop_address     935 non-null    object 
 7   day_of_purchase  935 non-null    object 
 8   load_date        935 non-null    object 
 9   category_id      935 non-null    int64  
dtypes: float64(1), int64(3), object(6)
memory usage: 73.2+ KB


None

'Обработка данных из Goods_2024.csv'

,bill_id,name,price,quantity,full_cost,shop_name,shop_address,day_of_purchase,load_date,category_id
0,896b7f48-a3a9-45d6-8723-8090c60a29b5,САЛАТ Фриллис в горшочке,6499,1.0,6499,Магазин Магнит Рента,"108801,Россия,город федерального значения Моск...",2024-01-15 11:14:00,2024-01-19 00:00:00,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55 entries, 0 to 54
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   bill_id          55 non-null     object 
 1   name             55 non-null     object 
 2   price            55 non-null     int64  
 3   quantity         55 non-null     float64
 4   full_cost        55 non-null     int64  
 5   shop_name        55 non-null     object 
 6   shop_address     55 non-null     object 
 7   day_of_purchase  55 non-null     object 
 8   load_date        55 non-null     object 
 9   category_id      55 non-null     int64  
dtypes: float64(1), int64(3), object(6)
memory usage: 4.4+ KB


None

# TODO, тут дальше песочница для метода разметки категорий

In [16]:
# Чай/кофе
CAT_1_1 = ['батон', 'нарезной', 'хлеб']
# Хлеб
CAT_1_2 = []
# Хлебобулочные продукты
CAT_1_3 = []
# Кондитерские изделия
CAT_1_4 = []
# Сладости и гадости
CAT_1_5 = []
# Пищевые жиры
CAT_1_6 = []
# Яйца
CAT_1_7 = []
# Мясо и мясопродукты
CAT_1_8 = []
# Рыба и рыбопродукты
CAT_1_9 = []
# Молоко и молочные продукты
CAT_1_10 = []
# Крупы
CAT_1_11 = []
# Макаронные изделия
CAT_1_12 = []
# Зелень
CAT_1_13 = []
# Овощи
CAT_1_14 = []
# Фрукты и ягоды
CAT_1_15 = []
# Алкоголь
CAT_1_16 = []
# Орехи
CAT_1_17 = []
# Грибы
CAT_1_18 = []
# Консервы
CAT_1_19 = []


In [17]:
any(word in 'Батон Нарезной из пш в/с 400г не упак(КБК Чер   '.lower() for word in CAT_1_1)

True

In [18]:
# TODO в будущем сюда модель классификации впихнуть примерно как в https://habr.com/ru/articles/430216/
def detect_category(good_name):
    name = good_name.lower()
    if any(word in name for word in CAT_1_1):
        return 'Чай/кофе'
    elif name in CAT_1_2:
        return 'Хлеб'
    elif name in CAT_1_3:
        return 'Хлебобулочные продукты'
    elif name in CAT_1_4:
        return 'Кондитерские изделия'
    elif name in CAT_1_5:
        return 'Сладости и гадости'
    elif name in CAT_1_6:
        return 'Пищевые жиры'
    elif name in CAT_1_7:
        return 'Яйца'
    elif name in CAT_1_8:
        return 'Мясо и мясопродукты'
    elif name in CAT_1_9:
        return 'Рыба и рыбопродукты'
    elif name in CAT_1_10:
        return 'Молоко и молочные продукты'
    elif name in CAT_1_11:
        return 'Крупы'
    elif name in CAT_1_12:
        return 'Макаронные изделия'
    elif name in CAT_1_13:
        return 'Зелень'
    elif name in CAT_1_14:
        return 'Овощи'
    elif name in CAT_1_15:
        return 'Фрукты и ягоды'
    elif name in CAT_1_16:
        return 'Орехи'
    elif name in CAT_1_17:
        return 'Грибы'
    elif name in CAT_1_18:
        return 'Консервы'
    elif name in CAT_1_19:
        return 'Алкоголь'
    else:
        return 'Другое'

In [19]:
full_df['cat_test'] = full_df.apply(lambda x: detect_category(x.name))

In [20]:
# full_df['cat_test']

In [21]:
pd.options.display.max_rows = 1000
full_df['name'].value_counts()

Батон Нарезной из пш в/с 400г не упак(КБК Чер                       29
ЛУК репчатый 1кг                                                    19
ОГУРЦЫ среднеплодные пупырчатые 1кг                                 17
АПЕЛЬСИНЫ 1кг                                                       16
ТОМАТЫ 1кг                                                          15
ЯБЛОКИ новый урожай 1кг                                             13
СЕЛО ЗЕЛЕНОЕ Молоко паст 3,2% 2кг пл/бут                            11
ТС2 Сахар-песок белый кристаллический фас 1кг                       10
Филе окорок свин                                                    10
ОГУРЦЫ короткоплодные пупырчатые 1кг                                 9
КАРТОФЕЛЬ фасованный цена за 1кг                                     9
Яйцо столовое СО 10шт бокс:20                                        9
Легкое говяжье                                                       9
ЭГО Молоко пастер пит 3,2% 1700мл пл/бут                             8
MAKFA 

In [22]:
# url = URL
# data = {
#     "token": TOKEN,
#     "qrraw": 't=20230721t1636&s=715.21&fn=9961440300305579&i=68881&fp=220365927&n=1',
# }
# r = requests.post(url, data=data)
# result = json.loads(r.text)
# result['data']['json']

## Раздел анализа данных